In [ ]:
import sys
import os
#sys.path.append('/home/sagemaker-user/portfolio_optimization')
import pandas as pd
import joblib
from ml import preprocess, train_expected_returns, predict_expected_returns

import os
from tqdm import tqdm
from importlib import reload
import config
import utils

reload(preprocess)
reload(train_expected_returns)
reload(predict_expected_returns)

## 1. TRAIN ALL MODELS FOR EXPECTED RETURNS

In [ ]:
print(files)
for f in tqdm(files):
    df = pd.read_csv(
        f"/Users/noahforougi/research/portfolio_optimization/data/clean/{f}"
    )
    df = preprocess.prepare_training_data(df)
    model_dict = train_expected_returns.train_expected_returns(df)
    clean_file = f.replace(".csv", "")
    joblib.dump(
        model_dict,
        f"/Users/noahforougi/research/portfolio_optimization/data/return_models/{clean_file}_er_models.pkl",
    )

## 2. PREDICTIONS FOR EXPECTED RETURNS

In [ ]:
dates = pd.date_range(config.START_DATE, config.END_DATE, freq="ME")
for f in tqdm(files):
    df = pd.read_csv(
        f"/Users/noahforougi/research/portfolio_optimization/data/clean/{f}"
    )
    model = f.replace(".csv", "_er_models.pkl")
    model_dict = joblib.load(
        f"/Users/noahforougi/research/portfolio_optimization/data/return_models/{model}"
    )
    df = preprocess.prepare_training_data(df)
    results = predict_expected_returns.expected_return_forecasts(
        df, model_dict=model_dict, dates=dates
    )
    results.to_csv(
        f"/Users/noahforougi/research/portfolio_optimization/data/output/er_forecasts_{f}",
        index=False,
    )